In [9]:
import tensorflow as tf
import tensorflow_datasets as tfds

import numpy as np

In [10]:
(ds_train, ds_test), ds_info = tfds.load(
    'mnist',
    split=['train', 'test'],
    shuffle_files=True,
    as_supervised=True,
    with_info=True,
)

In [11]:
def normalize_img(image, label):
  """Normalizes images: `uint8` -> `float32`."""
  return tf.cast(image, tf.float32) / 255., label

ds_train = ds_train.map(
    normalize_img, num_parallel_calls=tf.data.AUTOTUNE)
ds_train = ds_train.cache()
ds_train = ds_train.shuffle(ds_info.splits['train'].num_examples)
ds_train = ds_train.batch(128)
ds_train = ds_train.prefetch(tf.data.AUTOTUNE)

In [12]:
ds_test = ds_test.map(
    normalize_img, num_parallel_calls=tf.data.AUTOTUNE)
ds_test = ds_test.batch(128)
ds_test = ds_test.cache()
ds_test = ds_test.prefetch(tf.data.AUTOTUNE)

In [13]:
model = tf.keras.models.Sequential([
  tf.keras.layers.Flatten(input_shape=(28, 28)),
  tf.keras.layers.Dense(32, activation='relu'),
  tf.keras.layers.Dense(10)
])
model.compile(
    optimizer=tf.keras.optimizers.Adam(0.001),
    loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    metrics=[tf.keras.metrics.SparseCategoricalAccuracy()],
)

model.fit(
    ds_train,
    epochs=6,
    validation_data=ds_test,
)

Epoch 1/6
469/469 [==============================] - 1s 1ms/step - loss: 0.4812 - sparse_categorical_accuracy: 0.8701 - val_loss: 0.2664 - val_sparse_categorical_accuracy: 0.9228
Epoch 2/6
469/469 [==============================] - 1s 1ms/step - loss: 0.2410 - sparse_categorical_accuracy: 0.9316 - val_loss: 0.2145 - val_sparse_categorical_accuracy: 0.9377
Epoch 3/6
469/469 [==============================] - 1s 1ms/step - loss: 0.1977 - sparse_categorical_accuracy: 0.9433 - val_loss: 0.1838 - val_sparse_categorical_accuracy: 0.9464
Epoch 4/6
469/469 [==============================] - 1s 3ms/step - loss: 0.1705 - sparse_categorical_accuracy: 0.9506 - val_loss: 0.1627 - val_sparse_categorical_accuracy: 0.9527
Epoch 5/6
469/469 [==============================] - 0s 778us/step - loss: 0.1492 - sparse_categorical_accuracy: 0.9564 - val_loss: 0.1497 - val_sparse_categorical_accuracy: 0.9568
Epoch 6/6
469/469 [==============================] - 1s 2ms/step - loss: 0.1344 - sparse_categorical_ac

In [24]:
def generate_weights_header(model, file_path):
    with open(file_path, 'w') as file:
        file.write("#ifndef WEIGHTS_H\n")
        file.write("#define WEIGHTS_H\n\n")

        for layer in model.layers:
            if "dense" in layer.name:
                weights = layer.get_weights()
                input_size, output_size = weights[0].shape
                
                
                file.write(f"const float {layer.name}_weights[{input_size}][{output_size}] = {{\n")
                for row in weights[0]:
                    file.write("    {")
                    file.write(", ".join(str(val) for val in row))
                    file.write("},\n")
                file.write("};\n\n")
                
                file.write(f"const float {layer.name}_bias[{output_size}] = {{\n")
                file.write(", ".join(str(val) for val in weights[1]))
                file.write("};\n\n")

        file.write("#endif  // WEIGHTS_H\n")

In [25]:
generate_weights_header(model, '/home/sungjin/workspace/MLP_MNIST/weights.h')